In [5]:
import pandas as pd
import numpy as np

In [6]:
user_input_load_profile= pd.read_csv(r'SampleNEM12.csv',header=None)


In [7]:
user_input_load_profile

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,100,NEM12,2.01904E+11,TCAUSTM,CUSTAG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,4103060331,B1E1,B1,B1,N1,2.165404e+08,KWH,30.000,2.016121e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,300,20180416,0,0,0,0,0.000000e+00,0,0.000,0.000000e+00,...,0.000,0.000,0.000,0.000,0.000,0.000,A,NaN,NaN,2.018040e+13
3,300,20180417,0,0,0,0,0.000000e+00,0,0.000,0.000000e+00,...,0.000,0.000,0.000,0.000,0.000,0.000,A,NaN,NaN,2.018040e+13
4,300,20180418,0,0,0,0,0.000000e+00,0,0.000,0.000000e+00,...,0.000,0.000,0.000,0.000,0.000,0.000,A,NaN,NaN,2.018040e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,300,20190409,0.089,0.091,0.091,0.06,5.500000e-02,0.043,0.056,7.200000e-02,...,0.126,0.129,0.090,0.073,0.034,0.037,A,NaN,NaN,2.019040e+13
740,300,20190410,0.032,0.024,0.05,0.054,5.500000e-02,0.046,0.028,2.700000e-02,...,0.097,0.104,0.076,0.063,0.037,0.035,A,NaN,NaN,2.019040e+13
741,300,20190411,0.021,0.038,0.048,0.045,4.800000e-02,0.039,0.028,2.800000e-02,...,0.094,0.081,0.099,0.087,0.061,0.065,A,NaN,NaN,2.019040e+13
742,300,20190412,0.035,0.041,0.025,0.043,4.900000e-02,0.046,0.049,3.900000e-02,...,0.121,0.106,0.083,0.084,0.048,0.041,A,NaN,NaN,2.019040e+13


In [8]:
NEM12_1 = user_input_load_profile.copy()

Chunks = np.where((NEM12_1[NEM12_1.columns[0]] == 200)|(NEM12_1[NEM12_1.columns[0]] == 900))[0]
NEM12_2 = pd.DataFrame()
for i in range(0, len(Chunks)-1):
    if NEM12_1.iloc[Chunks[i], 4].startswith('E'):
        this_part = NEM12_1.iloc[Chunks[i] + 1: Chunks[i + 1], :].copy()
        this_part = this_part[this_part[this_part.columns[0]] == 300].copy()
        this_part2 = this_part.iloc[:, 2:50]
        this_part2 = this_part2.astype(float)
        if (this_part2[this_part2 < 0.01].count().sum() / this_part2.count().sum()) < 0.3: # assume for controlled load more 30% of data points are zero
            NEM12_2 = NEM12_1.iloc[Chunks[i] + 1: Chunks[i + 1], :].copy()
            NEM12_2.reset_index(inplace=True, drop=True)

NEM12_2 = NEM12_2[NEM12_2[NEM12_2.columns[0]] == 300].copy()
NEM12_2[NEM12_2.columns[1]] = NEM12_2[NEM12_2.columns[1]].astype(int).astype(str)

Nem12 = NEM12_2.iloc[:, 1:50].melt(id_vars=[1], var_name="HH", value_name="kWh") # it was 49.. need to check if Dan manually changed it
Nem12['HH'] = Nem12['HH']-1
Nem12['kWh'] = Nem12['kWh'].astype(float)

Nem12['Datetime'] = pd.to_datetime(Nem12[1], format='%Y%m%d') + pd.to_timedelta(Nem12['HH'] * 30, unit='m')
Nem12.sort_values('Datetime', inplace=True)
# Nem12_ = Nem12.groupby(['Datetime','HH']).sum().reset_index()
Nem12.reset_index(inplace=True, drop=True)
sample_load = Nem12[['Datetime', 'kWh']].copy()

In [9]:
sample_load

,Datetime,kWh
0,2018-04-16 00:30:00,0.029
1,2018-04-16 01:00:00,0.039
2,2018-04-16 01:30:00,0.046
3,2018-04-16 02:00:00,0.049
4,2018-04-16 02:30:00,0.051
...,...,...
17371,2019-04-12 22:00:00,0.106
17372,2019-04-12 22:30:00,0.083
17373,2019-04-12 23:00:00,0.084
17374,2019-04-12 23:30:00,0.048
